In [0]:
%run ../utils/common

In [0]:
dbutils.widgets.text("environment", "", "")
environment = dbutils.widgets.get("environment")

In [0]:
catalog_name = settings[environment]['catalog_name']

In [0]:
spark.sql(f"""
CREATE OR REPLACE TABLE {catalog_name}.udp_wcm_gold_vcm_dwh.d_promotion
TBLPROPERTIES (
  'DELTA.AUTOOPTIMIZE.OPTIMIZEWRITE' = 'TRUE',
  'DELTA.AUTOOPTIMIZE.AUTOCOMPACT' = 'TRUE'
)
AS
WITH avccmdm332 AS (
SELECT bbynr, prqnr, matnr , matquan , matunit , disctype , grpgnr 
FROM  {catalog_name}.udp_wcm_silver_sap_bw.sap_avccmdm332
QUALIFY ROW_NUMBER() OVER (PARTITION BY bbynr, prqnr, matnr ORDER BY end_date DESC) = 1
),
avccmdm342 AS (
SELECT 
 bbynr,
 status,
 aktnr ,
 buy_prqval_min,
 crdate as created_on
FROM  {catalog_name}.udp_wcm_silver_sap_bw.sap_avccmdm342
QUALIFY ROW_NUMBER() OVER (PARTITION BY bbynr ORDER BY end_date DESC) = 1
),
trt_promo AS (
SELECT rt_promo, 
txtmd 
FROM {catalog_name}.udp_wcm_silver_sap_bw.sap_trt_promo
QUALIFY ROW_NUMBER() OVER (PARTITION BY rt_promo ORDER BY end_date DESC) = 1
),
prt_promo AS (
SELECT rt_promo, 
_bic_zbbuytype, 
_bic_zc_bbtype, 
_bic_zc_bbprof, 
_bic_zk_fm_km, 
_bic_zk_fm_th, 
_bic_zbbprtven , 
rt_promoct 
FROM {catalog_name}.udp_wcm_silver_sap_bw.sap_prt_promo
QUALIFY ROW_NUMBER() OVER (PARTITION BY rt_promo ORDER BY end_date DESC) = 1
),
avccmdm321 AS (
  SELECT *
  FROM( SELECT * FROM {catalog_name}.udp_wcm_silver_sap_bw.sap_avccmdm321
  QUALIFY 
    ROW_NUMBER() OVER (
      PARTITION BY kappl, kschl, vkorg, vtweg, werks, bbynr, matnr, vrkme 
      ORDER BY proc_date DESC
    ) = 1
     )
   QUALIFY RANK() OVER (
      PARTITION BY kappl, bbynr, vkorg 
      ORDER BY DATE(proc_date) DESC
    ) = 1
)
,
avccmdm412 AS (
SELECT * EXCEPT (bbynr), LTRIM('0',bbynr) AS bbynr 
FROM {catalog_name}.udp_wcm_silver_sap_bw.sap_avccmdm412
QUALIFY ROW_NUMBER() OVER (PARTITION BY bbynr ORDER BY proc_date DESC) = 1
)
SELECT 
a.kschl AS discount_type,
a.vkorg AS sale_org,
a.vtweg AS distribution_channel,
a.werks AS store_id,
a.bbynr AS bonus_buy_id,
c.status AS bonus_buy_status,
c.created_on bonus_buy_created_on,
a.matnr AS discount_product_id,
a.vrkme AS discount_product_uom,
a.datab AS promo_start_date,
a.datbi AS promo_end_date,
a.knumh AS condition_rec_no,
a.vakey AS get_item_no,
a.rewref AS condition_reference_no,
a.bbyval AS bby_cond_value_fix,
a.bbyper AS bby_cond_value_percent,
a.bbycur AS bby_currency,
a.kpein AS price_unit,
a.kmein AS get_item_uom,
a.grpnr AS product_group_no,
a.stfkz AS scale_type_code,
a.mquan AS get_item_qty,
a.bbytype AS bby_type_code,
a.filgr AS site_group_code,
b.prqnr AS buy_item_no,
b.matnr AS buy_item_id,
b.grpgnr AS buy_item_grouping,
b.matquan AS buy_item_qty,
b.matunit AS buy_item_uom,
b.disctype AS buy_discount_type,
c.aktnr AS promotion_id,
d.txtmd AS promotion_name,
e.rt_promoct AS promotion_type,
c.buy_prqval_min AS total_bill_value,
e._bic_zc_bbtype AS bonus_buy_type,
e._bic_zbbuytype AS bonus_buy_note,
e._bic_zc_bbprof AS bonus_buy_profile,
e._bic_zk_fm_km AS zk_fm_km,
e._bic_zk_fm_th AS zk_fm_th,
e._bic_zbbprtven AS zbbprtven,
f.member_only,
f.customer_type,
f.limit_customer,
f.vendor_promotion_id,
f.pct_shared_by_vendor,
f.apply_pnl,
f.value_claimback,
f.form_claimback
FROM avccmdm321 a
LEFT JOIN avccmdm332 b ON a.bbynr = b.bbynr
LEFT JOIN avccmdm342 c ON a.bbynr = c.bbynr
LEFT JOIN trt_promo d ON c.aktnr = d.rt_promo
LEFT JOIN prt_promo e ON a.bbynr = e.rt_promo
LEFT JOIN avccmdm412 f ON a.bbynr = f.bbynr
""")